# Search

In [27]:
import weaviate
from ruia import Item, Spider, TextField, AttrField, Request

In [25]:
client = weaviate.Client("http://us-central-02.db.sybil.com:8080")  # Replace the URL with that of your Weaviate instance
initial_link = "https://wikipedia.org"


## spider

In [26]:
class HackerNewsItem(Item):
    target_item = TextField(css_select="tr.athing")
    title = TextField(css_select="a.storylink")
    url = AttrField(css_select="a.storylink", attr="href")

In [ ]:
class HackerNewsSpider(Spider):
    request_config = {"RETRIES": 3, "DELAY": 0, "TIMEOUT": 20}
    concurrency = 3
    start_urls = ["https://news.ycombinator.com"]

    async def parse(self, response):
        urls = [
            "https://news.ycombinator.com/news?p=1",
            "https://news.ycombinator.com/news?p=2",
        ]
        for index, url in enumerate(urls):
            yield Request(url, callback=self.parse_item, metadata={"index": index})

    async def parse_item(self, response):
        async for item in HackerNewsItem.get_items(html=await response.text()):
            yield item

    async def process_item(self, item):
        try:
            print(item.title, item.url)
        except Exception as e:
            self.logger.error(e)